<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/EDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive
 98% 673M/688M [00:07<00:00, 94.2MB/s]
100% 688M/688M [00:08<00:00, 90.0MB/s]


In [3]:
#Take app train data and load to DF
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
bur_data = pd.read_csv('/content/home-credit-default-risk/bureau.csv')

print(bur_data.shape)

(1716428, 17)


# Bureau_Balance

In [13]:
burbal_data = pd.read_csv('/content/home-credit-default-risk/bureau_balance.csv')

print(burbal_data.shape)

(27299925, 3)


In [90]:
burbal_data[burbal_data['SK_ID_BUREAU']==5715448]

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
5,5715448,-5,C
6,5715448,-6,C
7,5715448,-7,C
8,5715448,-8,C
9,5715448,-9,0


In [91]:
from pandas.core.dtypes.cast import Sized
#burbal_data1 = burbal_data[burbal_data['SK_ID_BUREAU']==5715448]

Summary_data1 = burbal_data.groupby('SK_ID_BUREAU', as_index=False).agg(
    BURBAL_CNT = ("SK_ID_BUREAU","count")
    , BURBAL_C_CNT = ("STATUS", lambda x: sum(x=='C'))
    , BURBAL_1_CNT = ("STATUS", lambda x: sum(x=='1'))
    , BURBAL_2_CNT = ("STATUS", lambda x: sum(x=='2'))
    , BURBAL_3_CNT = ("STATUS", lambda x: sum(x=='3'))
    , BURBAL_4_CNT = ("STATUS", lambda x: sum(x=='4'))
    , BURBAL_5_CNT = ("STATUS", lambda x: sum(x=='5'))
    , BURBAL_X_CNT = ("STATUS", lambda x: sum(x=='X'))
    , BURBAL_MTH_MIN = ("MONTHS_BALANCE", "min")
    , BURBAL_MTH_MAX = ("MONTHS_BALANCE", "max")
)

burbal_data1 = burbal_data[burbal_data['STATUS']=='1']
Summary_data2 = burbal_data1.groupby('SK_ID_BUREAU', as_index=False).agg(
    BURBAL_1_MTH_MIN = ("MONTHS_BALANCE", "min")
    , BURBAL_1_MTH_MAX = ("MONTHS_BALANCE", "max") )

Summary_data3 = Summary_data1.merge(Summary_data2,on='SK_ID_BUREAU',how='left',suffixes=('_1','_2'))

print(Summary_data1.shape)
print(Summary_data2.shape)
print(Summary_data3.shape)

(817395, 11)
(99142, 3)
(817395, 13)


In [92]:
bur_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              1716428 non-null  int64  
 1   SK_ID_BUREAU            1716428 non-null  int64  
 2   CREDIT_ACTIVE           1716428 non-null  object 
 3   CREDIT_CURRENCY         1716428 non-null  object 
 4   DAYS_CREDIT             1716428 non-null  int64  
 5   CREDIT_DAY_OVERDUE      1716428 non-null  int64  
 6   DAYS_CREDIT_ENDDATE     1610875 non-null  float64
 7   DAYS_ENDDATE_FACT       1082775 non-null  float64
 8   AMT_CREDIT_MAX_OVERDUE  591940 non-null   float64
 9   CNT_CREDIT_PROLONG      1716428 non-null  int64  
 10  AMT_CREDIT_SUM          1716415 non-null  float64
 11  AMT_CREDIT_SUM_DEBT     1458759 non-null  float64
 12  AMT_CREDIT_SUM_LIMIT    1124648 non-null  float64
 13  AMT_CREDIT_SUM_OVERDUE  1716428 non-null  float64
 14  CR

In [93]:
bur_data[bur_data['AMT_CREDIT_SUM_OVERDUE']>0].head(2)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
365,282297,5714943,Active,currency 1,-2712,2603,"-2,498.00",NaN,"4,590.00",0,"24,696.00",231.53,0.00,231.53,Consumer credit,-1077,NaN
785,408647,5715468,Active,currency 1,-313,6,233.00,NaN,NaN,0,"417,208.50","214,947.00",0.00,288.00,Consumer credit,-5,NaN


In [94]:
bur_data['CREDIT_CURRENCY'].value_counts()

currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64

In [95]:
bur_data_full = bur_data.merge(Summary_data3,on='SK_ID_BUREAU',how='left',suffixes=('_1','_2'))
print(bur_data_full.shape)

(1716428, 29)


In [98]:
bur_data_full.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1716428 entries, 0 to 1716427
Data columns (total 29 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              1716428 non-null  int64  
 1   SK_ID_BUREAU            1716428 non-null  int64  
 2   CREDIT_ACTIVE           1716428 non-null  object 
 3   CREDIT_CURRENCY         1716428 non-null  object 
 4   DAYS_CREDIT             1716428 non-null  int64  
 5   CREDIT_DAY_OVERDUE      1716428 non-null  int64  
 6   DAYS_CREDIT_ENDDATE     1610875 non-null  float64
 7   DAYS_ENDDATE_FACT       1082775 non-null  float64
 8   AMT_CREDIT_MAX_OVERDUE  591940 non-null   float64
 9   CNT_CREDIT_PROLONG      1716428 non-null  int64  
 10  AMT_CREDIT_SUM          1716415 non-null  float64
 11  AMT_CREDIT_SUM_DEBT     1458759 non-null  float64
 12  AMT_CREDIT_SUM_LIMIT    1124648 non-null  float64
 13  AMT_CREDIT_SUM_OVERDUE  1716428 non-null  float64
 14  CR

In [110]:
from pandas.core.dtypes.cast import Sized
#bur_data1 = bur_data[bur_data['SK_ID_CURR']==282297]

Summary_data = bur_data_full.groupby('SK_ID_CURR', as_index=False).agg(
    BUR_CNT = ("SK_ID_BUREAU","count")
    , BUR_ACTIVE_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Active'))
    , BUR_SOLD_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Sold'))
    , BUR_BAD_DEPT_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Bad debt'))
    , BUR_CUR_USED = ("CREDIT_CURRENCY","nunique")
    , BUR_CUR_OTHR = ("CREDIT_CURRENCY", lambda x: sum(x.isin(['currency 1']) == False))
    , BUR_CRD_TYPES = ("CREDIT_TYPE","nunique")
    , BUR_CRD_TYPES_OTHR=("CREDIT_TYPE", lambda x: sum(x.isin(['Consumer credit', 'Credit card'
                                                  , 'Car loan', 'Mortgage']) == False))
    , BUR_DAYS_CRD_VAR = ("DAYS_CREDIT", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_MIN = ("DAYS_CREDIT", "max")
    , BUR_OD_DAY_CNT = ("CREDIT_DAY_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_DAY_VAR = ("CREDIT_DAY_OVERDUE", lambda x:max(x) - min(x))
    , BUR_OD_DAY_MIN = ("CREDIT_DAY_OVERDUE", "min")
    , BUR_DAYS_CRD_ED_CNT1 = ("DAYS_CREDIT_ENDDATE", lambda x:sum(x>0))
    , BUR_DAYS_CRD_ED_CNT2 = ("DAYS_CREDIT_ENDDATE", lambda x:sum(x<0))
    , BUR_DAYS_ED_FACT_VAR = ("DAYS_ENDDATE_FACT", lambda x:max(x) - min(x))
    , BUR_OD_AMT_MAX_CNT = ("AMT_CREDIT_MAX_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_MAX_VAR = ("AMT_CREDIT_MAX_OVERDUE", lambda x:max(x) - min(x))
    , BUR_CRD_PRO_CNT = ("CNT_CREDIT_PROLONG", lambda x:sum(x>0))
    , BUR_CRD_PRO_VAR = ("CNT_CREDIT_PROLONG", lambda x:max(x) - min(x))
    , BUR_CRD_AMT_VAR = ("AMT_CREDIT_SUM", lambda x:max(x) - min(x))
    , BUR_CRD_DBT_CNT = ("AMT_CREDIT_SUM_DEBT", lambda x:sum(x>0))
    , BUR_CRD_DBT_MAX = ("AMT_CREDIT_SUM_DEBT", "max")
    , BUR_CRD_LMT_CNT = ("AMT_CREDIT_SUM_LIMIT", lambda x:sum(x>0))
    , BUR_CRD_LMT_MAX = ("AMT_CREDIT_SUM_LIMIT", "max")
    , BUR_CRD_LMT_VAR = ("AMT_CREDIT_SUM_LIMIT", lambda x:max(x) - min(x))
    , BUR_OD_AMT_SUM_CNT = ("AMT_CREDIT_SUM_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_SUM_VAR = ("AMT_CREDIT_SUM_OVERDUE", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_UPDT_VAR = ("DAYS_CREDIT_UPDATE", lambda x:max(x) - min(x))
    , BUR_DAYS_CRD_UPDT_MIN = ("DAYS_CREDIT_UPDATE", "max")
    , BUR_ANNUITY_VAR = ("AMT_CREDIT_SUM_DEBT", lambda x:max(x) - min(x))

    , BURBAL_CNT = ("BURBAL_CNT","sum")
    , BURBAL_C_CNT = ("BURBAL_C_CNT","sum")
    , BURBAL_1_CNT = ("BURBAL_1_CNT","sum")
    , BURBAL_2_CNT = ("BURBAL_2_CNT","sum")
    , BURBAL_3_CNT = ("BURBAL_3_CNT","sum")
    , BURBAL_4_CNT = ("BURBAL_4_CNT","sum")
    , BURBAL_5_CNT = ("BURBAL_5_CNT","sum")
    , BURBAL_X_CNT = ("BURBAL_X_CNT","sum")
    , BURBAL_MTH_MIN = ("BURBAL_MTH_MIN", "min")
    , BURBAL_MTH_MAX = ("BURBAL_MTH_MAX", "max")
    , BURBAL_1_MTH_MIN = ("BURBAL_1_MTH_MIN", "min")
    , BURBAL_1_MTH_MAX = ("BURBAL_1_MTH_MAX", "max")

)
Summary_data.shape

(305811, 44)

In [112]:
 Summary_data[Summary_data['BUR_OD_AMT_SUM_VAR']>0][['BUR_CRD_AMT_VAR','BUR_CRD_DBT_MAX','BUR_CRD_LMT_MAX','BUR_OD_AMT_SUM_VAR','BUR_OD_AMT_MAX_VAR']]

,BUR_CRD_AMT_VAR,BUR_CRD_DBT_MAX,BUR_CRD_LMT_MAX,BUR_OD_AMT_SUM_VAR,BUR_OD_AMT_MAX_VAR
103,"143,910.00","108,837.00",0.00,166.50,NaN
139,"961,384.50","850,653.00",0.00,13.50,"16,320.51"
298,"287,874.54","204,894.00",0.00,459.00,"29,654.01"
306,"886,500.00","430,722.00",0.00,"19,359.00",NaN
475,"166,500.00","175,464.00",0.00,360.00,NaN
...,...,...,...,...,...
305464,"1,398,496.50","937,233.00","50,700.64",522.00,"5,490.00"
305470,"844,024.50","584,181.00",0.00,477.00,NaN
305580,"33,997.50",0.00,0.00,9.00,NaN
305687,"112,500.00","69,404.67",0.00,13.50,NaN


In [113]:
combined_data=app_data[['SK_ID_CURR','TARGET']].merge(Summary_data,on='SK_ID_CURR',how='left',suffixes=('_1','_2'))
combined_data.shape

(307511, 45)

In [123]:
combined_data['BUR_MAX_OD_PER_OD'] = combined_data['BUR_OD_AMT_MAX_VAR']/combined_data['BUR_OD_AMT_SUM_VAR']
combined_data['BUR_BAD_DEPT_RTO'] = combined_data['BUR_BAD_DEPT_CNT']/combined_data['BUR_CNT']
combined_data['BUR_CRD_LMT_TO_DBT'] = combined_data['BUR_CRD_LMT_MAX']/combined_data['BUR_CRD_DBT_MAX']
combined_data['BUR_CRD_TO_DBT'] = combined_data['BUR_CRD_AMT_VAR']/combined_data['BUR_CRD_DBT_MAX']


In [127]:
Summary_data['BUR_MAX_OD_PER_OD'] = Summary_data['BUR_OD_AMT_MAX_VAR']/Summary_data['BUR_OD_AMT_SUM_VAR']
Summary_data['BUR_BAD_DEPT_RTO'] = Summary_data['BUR_BAD_DEPT_CNT']/Summary_data['BUR_CNT']
Summary_data['BUR_CRD_LMT_TO_DBT'] = Summary_data['BUR_CRD_LMT_MAX']/Summary_data['BUR_CRD_DBT_MAX']
Summary_data['BUR_CRD_TO_DBT'] = Summary_data['BUR_CRD_AMT_VAR']/Summary_data['BUR_CRD_DBT_MAX']

Summary_data.to_csv('/content/home-credit-default-risk/bureau_summary.csv', index=False)


In [124]:
# correlation with Target - target value
corr_features = pd.DataFrame()
corr_features['FI'] = combined_data.corr()['TARGET']
corr_features['FI_MOD']  = corr_features['FI'].abs()
print(corr_features.sort_values(by='FI_MOD',ascending=False).head(50)) 
#corr_features.style.background_gradient(cmap='viridis')

                         FI  FI_MOD
TARGET                 1.00    1.00
BUR_CRD_DBT_CNT        0.08    0.08
BURBAL_MTH_MIN         0.07    0.07
BUR_ACTIVE_CNT         0.07    0.07
BURBAL_1_MTH_MAX       0.06    0.06
BUR_DAYS_CRD_ED_CNT1   0.05    0.05
BURBAL_1_MTH_MIN       0.05    0.05
BUR_DAYS_CRD_MIN       0.05    0.05
BUR_OD_AMT_SUM_CNT     0.04    0.04
BUR_OD_DAY_CNT         0.04    0.04
BUR_DAYS_CRD_VAR      -0.04    0.04
BUR_MAX_OD_PER_OD     -0.04    0.04
BUR_DAYS_ED_FACT_VAR  -0.04    0.04
BUR_OD_AMT_MAX_CNT     0.04    0.04
BUR_CRD_LMT_CNT        0.04    0.04
BUR_DAYS_CRD_ED_CNT2  -0.04    0.04
BUR_DAYS_CRD_UPDT_VAR -0.03    0.03
BUR_CRD_TYPES_OTHR     0.03    0.03
BUR_DAYS_CRD_UPDT_MIN  0.03    0.03
BURBAL_C_CNT          -0.02    0.02
BURBAL_1_CNT           0.02    0.02
BUR_CRD_AMT_VAR       -0.02    0.02
BURBAL_CNT            -0.01    0.01
BUR_SOLD_CNT           0.01    0.01
BUR_OD_AMT_SUM_VAR     0.01    0.01
BUR_CRD_LMT_MAX       -0.01    0.01
BURBAL_4_CNT           0.01 